In [ ]:
# @title 1. Environment Setup & Model Download
import os
import shutil
import sys

# --- 1. Install Dependencies ---
print("📦 Installing libraries...")
!pip install -q flask flask-cors pyngrok speciesnet

# --- 2. Setup Cache Paths (Prevents permission errors) ---
CACHE_DIR = "/content/.cache/speciesnet"
os.makedirs(CACHE_DIR, exist_ok=True)
os.environ["XDG_CACHE_HOME"] = CACHE_DIR
os.environ["SPECIESNET_CACHE"] = CACHE_DIR
os.environ["TORCH_HOME"] = CACHE_DIR

# --- 3. Download SpeciesNet Model ---
# We define where the model lives
MODEL_FOLDER = "/content/speciesnet_model"
ARCHIVE_PATH = "/content/speciesnet_archive.tar.gz"

# Clean up previous runs to avoid corruption
if os.path.exists(MODEL_FOLDER):
    shutil.rmtree(MODEL_FOLDER)
os.makedirs(MODEL_FOLDER, exist_ok=True)

print("⬇️ Downloading SpeciesNet Model (approx 200MB)...")
!wget -q --show-progress -O "$ARCHIVE_PATH" "https://www.kaggle.com/api/v1/models/google/speciesnet/pyTorch/v4.0.2a/1/download"

print("📦 Extracting model...")
!tar -xzf "$ARCHIVE_PATH" -C "$MODEL_FOLDER"

# Create folders for the web app
os.makedirs('templates', exist_ok=True)
os.makedirs('uploads', exist_ok=True)
os.makedirs('temp_inference', exist_ok=True)

print("✅ Setup Complete! Model is ready at:", MODEL_FOLDER)

📦 Installing libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 956.3/956.3 kB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 12.

In [ ]:
!pip install flask_sqlalchemy

In [ ]:
import os
import shutil
import base64
import json
import subprocess
import sys
import time
import uuid
import torch
import gc
import cv2
from werkzeug.utils import secure_filename
from flask import Flask, render_template, request, jsonify, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
from threading import Lock
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

# App Configuration
app = Flask(__name__)
CORS(app)

NGROK_AUTH_TOKEN = "TOKEN"
MODEL_NAME = "/content/speciesnet_model"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Directory Setup
BASE_DIR = os.getcwd()
DETECTIONS_DIR = os.path.join(BASE_DIR, "static", "detections")
VIDEO_DIR = os.path.join(BASE_DIR, "uploads", "videos")

os.makedirs(DETECTIONS_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

print(f"📂 IMAGES WILL BE SAVED TO: {DETECTIONS_DIR}")

# Database Config
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///species_data.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

video_processor = None

# Database Models
class VideoRecord(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    filename = db.Column(db.String(255), nullable=False)
    filepath = db.Column(db.String(255), nullable=False)
    upload_time = db.Column(db.DateTime, default=datetime.utcnow)
    processed = db.Column(db.Boolean, default=False)
    detections = db.relationship('DetectionResult', backref='video', lazy=True)

class DetectionResult(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    video_id = db.Column(db.Integer, db.ForeignKey('video_record.id'), nullable=False)
    species = db.Column(db.String(100), nullable=False)
    confidence = db.Column(db.Float, nullable=False)
    timestamp_in_video = db.Column(db.Float, nullable=False)
    image_url = db.Column(db.String(255), nullable=True)

with app.app_context():
    db.create_all()

# Video Processing Logic
class BatchVideoProcessor:
    def __init__(self, model_manager):
        self.model_manager = model_manager

    def process_video_batched(self, video_path, batch_size=8, sample_fps=1, min_confidence=0.3, country='IND'):
        print(f"\n🏆 PROCESSING VIDEO: {video_path}")

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_interval = int(max(1, fps / sample_fps))

        paths_buffer = []
        timestamps_buffer = []
        all_detections = []

        frame_count = 0
        processed_count = 0

        temp_dir = "temp_inference/frames"
        os.makedirs(temp_dir, exist_ok=True)

        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_count % frame_interval == 0:
                    current_time_sec = frame_count / fps if fps > 0 else 0
                    frame_name = f"frame_{processed_count}.jpg"
                    frame_path = os.path.join(temp_dir, frame_name)
                    cv2.imwrite(frame_path, frame)

                    paths_buffer.append(frame_path)
                    timestamps_buffer.append(current_time_sec)

                    if len(paths_buffer) >= batch_size:
                        batch_detections = self._process_batch(paths_buffer, timestamps_buffer, country, min_confidence)
                        all_detections.extend(batch_detections)

                        for p in paths_buffer:
                            if os.path.exists(p): os.remove(p)
                        paths_buffer = []
                        timestamps_buffer = []

                        print(f"⏳ Progress: {frame_count/total_frames:.1%}")

                frame_count += 1
                processed_count += 1

            if paths_buffer:
                batch_detections = self._process_batch(paths_buffer, timestamps_buffer, country, min_confidence)
                all_detections.extend(batch_detections)
                for p in paths_buffer:
                    if os.path.exists(p): os.remove(p)

        finally:
            cap.release()

        print(f"✅ Video processing complete. Found {len(all_detections)} detections.")
        return all_detections

    def _process_batch(self, filepaths, timestamps, country, min_confidence):
        if not self.model_manager.model:
            return []

        try:
            path_to_time = {fp: ts for fp, ts in zip(filepaths, timestamps)}

            result = self.model_manager.model.predict(
                filepaths=filepaths,
                country=country,
                batch_size=len(filepaths)
            )

            valid_detections = []
            predictions = result.get('predictions', {})

            if isinstance(predictions, list):
                iterator = zip(filepaths, predictions)
                is_dict_format = False
            elif isinstance(predictions, dict):
                iterator = predictions.items()
                is_dict_format = True
            else:
                return []

            for item in iterator:
                path_key, pred_data = item if not is_dict_format else item

                if not pred_data: continue

                class_data = pred_data.get("classifications", {})
                if not class_data: continue

                top_score = class_data.get("scores", [0])[0]

                if top_score >= min_confidence:
                    top_class = class_data.get("classes", ["Unknown"])[0]

                    if ";" in top_class:
                        parts = [p.strip() for p in top_class.split(";") if p.strip()]
                        common_name = parts[-1].title()
                    else:
                        common_name = top_class.title()

                    time_sec = path_to_time.get(path_key, 0)
                    m, s = divmod(time_sec, 60)
                    h, m = divmod(m, 60)
                    timestamp_str = "{:02d}:{:02d}:{:02d}".format(int(h), int(m), int(s))

                    unique_name = f"det_{uuid.uuid4().hex[:8]}.jpg"
                    save_path = os.path.join(DETECTIONS_DIR, unique_name)

                    if os.path.exists(path_key):
                        img = cv2.imread(path_key)
                        if img is not None:
                            # Optimize image size
                            target_width = 640
                            h, w = img.shape[:2]
                            if w > target_width:
                                scale = target_width / w
                                new_h = int(h * scale)
                                img = cv2.resize(img, (target_width, new_h), interpolation=cv2.INTER_AREA)

                            cv2.imwrite(save_path, img)
                            image_url = f"/static/detections/{unique_name}"
                        else:
                            image_url = None
                        os.remove(path_key)
                    else:
                        image_url = None

                    valid_detections.append({
                        "species": common_name,
                        "confidence": float(top_score),
                        "timestamp": time_sec,
                        "timestamp_str": timestamp_str,
                        "image_url": image_url
                    })

            return valid_detections

        except Exception as e:
            print(f"❌ Batch error: {e}")
            import traceback
            traceback.print_exc()
            return []

# Singleton Model Manager
class ModelManager:
    _instance = None
    _lock = Lock()

    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
                    cls._instance.model = None
                    cls._instance.initialized = False
        return cls._instance

    def initialize(self):
        if self.initialized:
            return self.model is not None

        print("\n" + "=" * 60)
        print("🧠 LOADING SPECIESNET MODEL...")
        print(f"📍 Device: {DEVICE}")
        print("=" * 60)

        try:
            from speciesnet import SpeciesNet
            start = time.time()
            self.model = SpeciesNet(
                model_name=MODEL_NAME,
                components='all',
                geofence=True,
                multiprocessing=False
            )
            print(f"✅ Model loaded in {time.time() - start:.1f}s")
            self._warmup()
            self.initialized = True
            return True
        except Exception as e:
            print(f"❌ Model load error: {e}")
            self.initialized = True
            return False

    def _warmup(self):
        print("🔥 Warming up GPU...")
        try:
            from PIL import Image
            import numpy as np
            dummy_path = "/tmp/warmup.jpg"
            Image.fromarray(np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)).save(dummy_path)
            _ = self.model.predict(filepaths=[dummy_path], country="IND", run_mode='single_thread', progress_bars=False)
            print("🔥 Warmup complete")
            os.remove(dummy_path)
        except Exception as e:
            print(f"⚠️ Warmup failed: {e}")

    def predict(self, image_path, country="IND"):
        if self.model is None: return None
        with self._lock:
            try:
                start = time.time()
                result = self.model.predict(
                    filepaths=[image_path],
                    country=country,
                    run_mode='single_thread',
                    batch_size=1,
                    progress_bars=False
                )
                print(f"⚡ Inference: {time.time()-start:.2f}s")
                return result
            except Exception as e:
                print(f"❌ Prediction error: {e}")
                return None

model_manager = ModelManager()

# Global State
last_status = {
    "motion": 0, "tilt": 0.0, "gunshot": 0, "temp": None,
    "free_heap": None, "min_heap": None, "rssi": None, "uptime": None
}
last_seen = 0
gunshot_timestamp = 0

# Helper Functions
def run_speciesnet_inference(image_path):
    print(f"🧠 Processing: {image_path}")
    if model_manager.model is not None:
        result = model_manager.predict(image_path)
        if result: return parse_result(result, image_path)
    return run_subprocess_inference(image_path)

def parse_result(data, image_path=None):
    if not data: return None
    try:
        predictions = data.get('predictions', {})
        pred_item = None
        if isinstance(predictions, dict):
            if image_path:
                for key, value in predictions.items():
                    if os.path.basename(image_path) in key:
                        pred_item = value; break
            if pred_item is None and predictions: pred_item = next(iter(predictions.values()))
        elif isinstance(predictions, list) and predictions:
            pred_item = predictions[0]

        if not pred_item: return None
        class_data = pred_item.get("classifications", {})
        class_list = class_data.get("classes", [])
        score_list = class_data.get("scores", [])
        if not class_list: return None

        top_raw = class_list[0]
        if ";" in top_raw:
            parts = [p.strip() for p in top_raw.split(";") if p.strip()]
            species = parts[-1].title() if parts else "Unknown"
            scientific = parts[-2] if len(parts) >= 2 else species
        else:
            species = top_raw.title(); scientific = top_raw

        return {"species": species, "scientific_name": scientific, "confidence": score_list[0]}
    except Exception: return None

def run_subprocess_inference(image_path):
    return None

def handle_amb82_video(video_file, sample_fps=1, min_conf=0.3, country='IND'):
    filename = secure_filename(f"amb82_{int(time.time())}.mp4")
    file_path = os.path.join(VIDEO_DIR, filename)
    video_file.save(file_path)

    new_video = VideoRecord(filename=filename, filepath=file_path)
    db.session.add(new_video)
    db.session.commit()

    print(f"💾 Video saved to DB with ID: {new_video.id}")

    global video_processor
    if video_processor is None:
        video_processor = BatchVideoProcessor(model_manager)

    try:
        detections = video_processor.process_video_batched(
            file_path,
            batch_size=8,
            sample_fps=sample_fps,
            min_confidence=min_conf,
            country=country
        )

        if detections:
            for d in detections:
                result = DetectionResult(
                    video_id=new_video.id,
                    species=d['species'],
                    confidence=d['confidence'],
                    timestamp_in_video=d['timestamp'],
                    image_url=d.get('image_url')
                )
                db.session.add(result)

        new_video.processed = True
        db.session.commit()
        print("✅ Database updated with results.")

        return {
            "success": True, "video_id": new_video.id,
            "count": len(detections), "results": detections
        }

    except Exception as e:
        print(f"❌ Processing Error: {e}")
        return {"success": False, "error": str(e)}

# Routes
@app.route('/')
def index(): return render_template('index.html')

@app.route('/sensor')
def sen(): return render_template('sensor.html')

@app.route('/dashboard')
def dashboard(): return render_template('amb82_dashboard.html')

@app.route('/api/detect', methods=['POST'])
def detect():
    try:
        data = request.json
        img_data = data.get('image')
        if not img_data: return jsonify(success=False), 400
        _, encoded = img_data.split(",", 1)
        binary = base64.b64decode(encoded)
        filename = f"upload_{int(time.time())}.jpg"
        filepath = os.path.join("uploads", filename)
        with open(filepath, "wb") as f: f.write(binary)

        result = run_speciesnet_inference(filepath)
        if result: return jsonify({"success": True, **result})
        return jsonify(success=False), 500
    except Exception as e: return jsonify(success=False, error=str(e)), 500

@app.route('/update', methods=['POST'])
def update():
    global last_status, last_seen, gunshot_timestamp
    data = request.json or {}
    print(f"📨 ESP: {data}")
    last_seen = time.time()
    for key in last_status:
        if key in data:
            last_status[key] = data[key]
            if key == "gunshot" and data[key] == 1: gunshot_timestamp = time.time()
    return {"status": "ok"}

@app.route('/status')
def status():
    t = time.time()
    return jsonify({
        **last_status,
        "esp_online": (t - last_seen) < 10,
        "gunshot": 1 if (t - gunshot_timestamp) < 5 else 0,
        "model_loaded": model_manager.model is not None
    })

@app.route('/health')
def health():
    gpu = {}
    if torch.cuda.is_available():
        gpu = {"gpu": torch.cuda.get_device_name(0), "mem": round(torch.cuda.memory_allocated()/1024**2, 1)}
    return jsonify({"status": "ok", "device": DEVICE, **gpu})

@app.route('/api/video/upload', methods=['POST'])
def upload_video():
    if 'video' not in request.files: return jsonify(success=False), 400
    video_file = request.files['video']
    if video_file.filename == '': return jsonify(success=False), 400

    country = request.form.get('country', 'IND')
    response_mode = request.args.get('mode', 'simple')

    full_data = handle_amb82_video(video_file, country=country)

    if response_mode == 'simple':
        return jsonify({"success": True, "status": "Ack", "id": full_data.get('video_id')}), 200
    else:
        return jsonify(full_data), 200

@app.route('/api/history')
def get_history():
    videos = VideoRecord.query.order_by(VideoRecord.upload_time.desc()).all()
    output = []
    for v in videos:
        detections = [
            {"species": d.species, "time": d.timestamp_in_video, "image_url": d.image_url}
            for d in v.detections
        ]
        output.append({
            "id": v.id, "filename": v.filename,
            "time": v.upload_time, "detections": detections
        })
    return jsonify(output)

@app.route('/uploads/videos/<path:filename>')
def serve_video(filename):
    return send_from_directory(VIDEO_DIR, filename)

@app.route('/static/detections/<path:filename>')
def serve_detections(filename):
    return send_from_directory(DETECTIONS_DIR, filename)

@app.route('/api/video/process_url', methods=['POST'])
def process_video_url():
    global video_processor
    data = request.json
    video_path = data.get('video_path')
    if not video_path or not os.path.exists(video_path): return jsonify(success=False), 400
    if video_processor is None: video_processor = BatchVideoProcessor(model_manager)
    try:
        dets = video_processor.process_video_batched(video_path, batch_size=8, country=data.get('country', 'IND'))
        return jsonify({"success": True, "detections": dets})
    except Exception as e: return jsonify(success=False, error=str(e)), 500

# Main Execution
if __name__ == '__main__':
    print("\n" + "🚀" * 25)
    print("STARTING SPECIESNET SERVER")
    print("🚀" * 25 + "\n")

    if model_manager.initialize():
        print("\n✅ MODEL READY!")
    else:
        print("\n⚠️ Model failed, checking fallback")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

    ngrok.kill()
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(5000)

    print(f"\n{'='*60}")
    print(f"🌐 PUBLIC URL: {public_url}")
    print(f"{'='*60}\n")

    app.run(host='0.0.0.0', port=5000, threaded=True)

📂 IMAGES WILL BE SAVED TO: /content/static/detections

🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
STARTING SPECIESNET SERVER
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀


🧠 LOADING SPECIESNET MODEL...
📍 Device: cuda
✅ Model loaded in 2.8s
🔥 Warming up GPU...
🔥 Warmup complete

✅ MODEL READY!

🌐 PUBLIC URL: NgrokTunnel: "https://7a823fb0e2fa.ngrok-free.app" -> "http://localhost:5000"

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


💾 Video saved to DB with ID: 1

🏆 PROCESSING VIDEO: /content/uploads/videos/amb82_1769587425.mp4
⏳ Progress: 65.3%


INFO:werkzeug:127.0.0.1 - - [28/Jan/2026 08:03:47] "POST /api/video/upload HTTP/1.1" 200 -


✅ Video processing complete. Found 10 detections.
✅ Database updated with results.
💾 Video saved to DB with ID: 2

🏆 PROCESSING VIDEO: /content/uploads/videos/amb82_1769587490.mp4
⏳ Progress: 65.3%


...

📨 ESP: {'motion': 0, 'tilt': 0.25, 'gunshot': 0, 'ratio': 0.0, 'zcr': 0}


INFO:werkzeug:127.0.0.1 - - [28/Jan/2026 08:54:51] "POST /update HTTP/1.1" 200 -


📨 ESP: {'motion': 1, 'tilt': 0.15, 'gunshot': 0, 'ratio': 0.0, 'zcr': 0}


INFO:werkzeug:127.0.0.1 - - [28/Jan/2026 08:54:53] "POST /update HTTP/1.1" 200 -


📨 ESP: {'motion': 0, 'tilt': 0.22, 'gunshot': 0, 'ratio': 0.0, 'zcr': 0}
